## Imports

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow import keras
from tensorflow.python.client import device_lib
from keras.models import Model
import boto3
from time import time
import os
import io
from tqdm import tqdm
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from numpy import sqrt
from numpy import argmax
import seaborn as sns
import math
import sklearn
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from abc import ABC, abstractmethod
import csv

## Classes to handle dataset images

In [2]:
class DatasetHandler(ABC):
    def __init__(self,
                 model_path,
                 obstacle_dataset,
                 no_obstacle_dataset,
                 img_width=200,
                 img_height=200,
                 batch_size=32,
                 rand_images=True,
                 num_rand_images=10,
                 thresh_csv_fname=None,
                 metrics_csv_fname=None):
        
        self.model_path = model_path
        self.obstacle_dataset = obstacle_dataset
        self.no_obstacle_dataset = no_obstacle_dataset
        self.dataset = '/'.join(self.obstacle_dataset.split('/')[:-1])
        self.img_width = img_width
        self.img_height = img_height
        self.obstacle_images = []
        self.no_obstacle_images = []
        self._update_image_lists = False
        self.rand_images = rand_images
        
        if rand_images:
            self.batch_size = min(batch_size, num_rand_images*2)
        else:
            self.batch_size = batch_size
            
        if rand_images:
            self.num_rand_images = num_rand_images
            self.num_obstacles = num_rand_images
            self.num_no_obstacles = num_rand_images
            self.num_images = num_rand_images*2
        else:
            # Will be determined while reading all images from dataset
            self.num_obstacles = 0
            self.num_no_obstacles = 0
            self.num_images = 0
            
        if rand_images:
            self.obstacle_image_names = self._get_random_dataset_image_names(self.obstacle_dataset)
            self.no_obstacle_image_names = self._get_random_dataset_image_names(self.no_obstacle_dataset)
        else:
            (self.obstacle_image_names,
            self.num_obstacles) = self._get_all_dataset_image_names(self.obstacle_dataset)
            (self.no_obstacle_image_names,
            self.num_no_obstacles) = self._get_all_dataset_image_names(self.no_obstacle_dataset)
            
        self.model_score = ModelScore()
            
        if thresh_csv_fname:
            thresh_csv_path = os.path.join(model_path, thresh_csv_fname)
            self.f_thresh_csv = open(thresh_csv_path, 'w')
            self.thresh_csv_writer = csv.writer(self.f_thresh_csv)
            
            self.thresh_csv_header = ['model',
                                      'dataset',
                                      'Max Lost Ratio',
                                      'Lower Threshold',
                                      'Upper Threshold',
                                      'Total Predictions',
                                      'TP',
                                      '% TP',
                                      'TN',
                                      '% TN',
                                      'FP',
                                      '% FP',
                                      'FN',
                                      '% FN',
                                      'Not Decided',
                                      '% Not Decided',
                                      'True Not Decided',
                                      '% True Not Decided']
            
            self.thresh_csv_writer.writerow(self.thresh_csv_header)
            
        if metrics_csv_fname:
            metrics_csv_path = os.path.join(model_path, metrics_csv_fname)
            self.f_metrics_csv = open(metrics_csv_path, 'w')
            self.metrics_csv_writer = csv.writer(self.f_metrics_csv)

            metrics_prefixes = ['test', 'validation', '']
            metrics_separators = [' ', ' ', '']
            metrics_functions = ['auc', 'recall', 'specifity']
            
            self.metrics_csv_header = ['Trial Name']
            self.metrics_csv_header.append('Model Score')
            self.metrics_csv_header.append('Lost True Values %')
            self.metrics_csv_header.append('Lower Threshold')
            self.metrics_csv_header.append('Upper Threshold')
            self.metrics_csv_header.append('% FP')
            self.metrics_csv_header.append('% FN')
            
            for i, metrics_prefix in enumerate(metrics_prefixes):
                for metrics_function in metrics_functions:
                    metric_name = metrics_separators[i].join([metrics_prefix, metrics_function])
                    self.metrics_csv_header.append('max ' + metric_name)
            
            self.metrics_csv_writer.writerow(self.metrics_csv_header)

    def write_csv(self, **args):        
        row = []
        keys = args.keys()
        
        if 'model' in keys:
            row.append(args['model'])
            
        row.append(self.dataset)
        
        for col in self.thresh_csv_header[2:]:
            if col in keys:
                row.append(args[col])
            else:
                row.append(None)
        
        self.thresh_csv_writer.writerow(row)
                
    @abstractmethod
    def _get_random_dataset_image_names(self, dataset):
        pass
    
    @abstractmethod
    def _get_all_dataset_image_names(self, dataset):
        pass
    
    @abstractmethod
    def _get_image(self, imname):
        pass
    
    def _preprocess_image(self, im):
        w = im.shape[1]
        im1 = im[:,:w//2]
        im2 = im[:,w//2:]
        sub = cv2.subtract(im1, im2)
        sub = sub.reshape(sub.shape[0], sub.shape[1], 1)
        arr = keras.preprocessing.image.smart_resize(sub,
                                                     (self.img_height, self.img_width),
                                                     interpolation='bilinear')
        arr /= 255.0
        return arr
    
    def image_generator(self):
        inputs = []
        outputs = [0]*self.num_no_obstacles + [1]*self.num_obstacles

        for image_name in tqdm(self.no_obstacle_image_names):
            im = self._get_image(image_name)
            if self._update_image_lists:
                self.no_obstacle_images.append(im)
            im = self._preprocess_image(im)
            inputs.append(im)

        for image_name in tqdm(self.obstacle_image_names):
            im = self._get_image(image_name)
            if self._update_image_lists:
                self.obstacle_images.append(im)
            im = self._preprocess_image(im)
            inputs.append(im)
            
        self._update_image_lists = True

        for i in range(0, self.num_images, self.batch_size):
            x = np.array(inputs[i:i+self.batch_size])
            y = np.array(outputs[i:i+self.batch_size])
            yield(x, y)
            
    @abstractmethod
    def get_metrics(self, model):
        pass
    
    @abstractmethod
    def get_predictions(self, model, color_mode='rgb'):
        pass
    
    def print_model_metrics(self, model, color_mode):
        metrics = self.get_metrics(model, color_mode)

        for name, value in zip(model.metrics_names, metrics):
            print(name, ': ', value)
            
    # Compute and plot confusion-matrix (TP, FP, TN, FN)
    def calc_cm(self,
                model_path,
                labels,
                predictions,
                p=0.5,
                max_lost_ratio=0,
                not_decided=0,
                trues=0,
                true_not_decided=0,
                lower_threshold=None,
                upper_threshold=None,
                print_params=True,
                plot=True,
                print_to_csv=False):

        cm = confusion_matrix(labels, predictions > p)        

        if plot:
            plt.figure(figsize=(5,5))
            sns.heatmap(cm, annot=True, fmt="d")
            if max_lost_ratio:
                plt.title('Confusion matrix @ max lost ratio {:.2f}'.format(max_lost_ratio))
            else:
                plt.title('Confusion matrix @ threshold {:.2f}'.format(p))
            plt.ylabel('Actual label')
            plt.xlabel('Predicted label')

        predicts = sum(sum(x) for x in cm)
        tot_predicts = predicts + not_decided
        
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]
        TP = cm[1][1]
        
        TN_percent = (TN/tot_predicts)*100
        FP_percent = (FP/tot_predicts)*100
        FN_percent = (FN/tot_predicts)*100
        TP_percent = (TP/tot_predicts)*100
        not_decided_percent = (not_decided/tot_predicts)*100
        true_not_decided_percent = (true_not_decided/trues)*100

        if print_params:
            print()
            if max_lost_ratio:
                print(f'Max Lost Ratio: {max_lost_ratio}')
            print(f'No Obstacles Detected (True Negatives): {TN} ({TN_percent:.2f}%)')
            print(f'No Obstacles Incorrectly Detected (False Positives): {FP} ({FP_percent:.2f}%)')
            print(f'Obstacles Missed (False Negatives): {FN} ({FN_percent:.2f}%)')
            print(f'Obstacles Detected (True Positives): {TP} ({TP_percent:.2f}%)')
            if not_decided:
                print(f'Not Decided: {not_decided} ({not_decided_percent:.2f}%)')
            if true_not_decided:
                print(f'True Not Decided: {true_not_decided} ({true_not_decided_percent:.2f}%)')
            print(f'Total Obstacles: {tot_predicts}')
        
        self.model_score.add_model_option(true_not_decided,
                                          true_not_decided_percent/100.0,
                                          FP,
                                          FP_percent/100.0,
                                          FN,
                                          FN_percent/100.0,
                                          lower_threshold,
                                          upper_threshold,
                                          predictions=predictions)
        
        if print_to_csv:
            # Prepare arguments for writing to csv
            args = {'model': model_path,
                    'Max Lost Ratio': max_lost_ratio,
                    'Lower Threshold': lower_threshold,
                    'Upper Threshold': upper_threshold,
                    'Total Predictions': tot_predicts,
                    'TP': TP,
                    '% TP': TP_percent,
                    'TN': TN,
                    '% TN': TN_percent,
                    'FP': FP,
                    '% FP': FP_percent,
                    'FN': FN,
                    '% FN': FN_percent,
                    'Not Decided': not_decided,
                    '% Not Decided': not_decided_percent,
                    'True Not Decided': true_not_decided,
                    '% True Not Decided': true_not_decided_percent}
            self.write_csv(**args)
        
        
    # Same as above, with consideration of lower and upper threshold 
    # For values between those thresholds "not decided" is returned, 
    # so it lowers the no. of FP and FN
    def calc_cm_considering_thresholds(self,
                                       model_path,
                                       labels,
                                       predictions,
                                       lower_threshold,
                                       upper_threshold,
                                       max_lost_ratio=0,
                                       threshold=0.5,
                                       print_params=True,
                                       plot=True,
                                       print_to_csv=False):
        no_obs_preds = predictions[:self.num_no_obstacles]
        no_obs_margin_preds = [p for p in no_obs_preds if p < lower_threshold or p > upper_threshold]
        no_obs_margin_labels = [0]*len(no_obs_margin_preds)
        true_no_obs_preds = [p for p in no_obs_preds if p <= 0.5]
        true_no_obs_not_decided_preds = [p for p in no_obs_preds if lower_threshold < p <= 0.5]
        obs_preds = predictions[self.num_no_obstacles:]
        obs_margin_preds = [p for p in obs_preds if p < lower_threshold or p > upper_threshold]
        obs_margin_labels = [1]*len(obs_margin_preds)
        true_obs_preds = [p for p in obs_preds if p >= 0.5]
        true_obs_not_decided_preds = [p for p in obs_preds if upper_threshold > p >= 0.5]
        margin_preds = no_obs_margin_preds + obs_margin_preds
        margin_preds = np.array(margin_preds, dtype=float)
        true_preds = true_no_obs_preds + true_obs_preds
        true_not_decided_preds = true_no_obs_not_decided_preds + true_obs_not_decided_preds
        margin_labels = no_obs_margin_labels + obs_margin_labels
        margin_labels = np.array(margin_labels, dtype=float)
        not_decided = len(predictions) - len(margin_preds)
        trues = len(true_preds)
        true_not_decided = len(true_not_decided_preds)

        self.calc_cm(model_path,
                     margin_labels,
                     margin_preds,
                     max_lost_ratio=max_lost_ratio,
                     not_decided=not_decided,
                     trues=trues,
                     true_not_decided=true_not_decided,
                     lower_threshold=lower_threshold,
                     upper_threshold=upper_threshold,
                     print_params=print_params,
                     plot=plot,
                     print_to_csv=print_to_csv)
    
    def display_false_negatives(self, predictions, filenames, lower_threshold):
        image_names = [s.split('/')[-1] for s in self.obstacle_image_names]
        false_negatives = [fname for i, fname in enumerate(filenames)\
                           if fname.split('/')[-1] in image_names\
                           and predictions[i] < lower_threshold]
        preds = [p for i, p in enumerate(predictions)\
                 if filenames[i].split('/')[-1] in image_names\
                 and p < lower_threshold]
        
        if 500 > len(false_negatives) > 1:
            num_images = len(false_negatives)
            _, axarr = plt.subplots(num_images, 1, figsize=(num_images, num_images))
            
            for i, fname in enumerate(false_negatives):
                imname = fname.split('/')[-1]
                impath = os.path.join(self.obstacle_dataset, imname)
                im = cv2.imread(impath)
                im1 = im[:, :, 0]
                im2 = im[:, :, 1]
                im12 = cv2.hconcat([im1, im2])
                axarr[i].imshow(im12, cmap='gray', vmin=0, vmax=255)
                axarr[i].set_title(str(preds[i]) + ' ' + imname + (' '*30), fontdict={'fontsize': 10})
            plt.show()
        elif false_negatives:
            fname = false_positives[0]
            imname = fname.split('/')[-1]
            impath = os.path.join(self.obstacle_dataset, imname)
            im = cv2.imread(impath)
            im1 = im[:, :, 0]
            im2 = im[:, :, 1]
            im12 = cv2.hconcat([im1, im2])
            plt.imshow(im12, cmap='gray', vmin=0, vmax=255)
            plt.title(str(preds[0])+ '\n' + imname)
            plt.show()   
            
    def display_false_positives(self, predictions, filenames, upper_threshold):
        image_names = [s.split('/')[-1] for s in self.no_obstacle_image_names]
        false_positives = [fname for i, fname in enumerate(filenames)\
                           if fname.split('/')[-1] in image_names\
                           and predictions[i] > upper_threshold]
        preds = [p for i, p in enumerate(predictions)\
                 if filenames[i].split('/')[-1] in image_names\
                 and p > upper_threshold]

        if 500 > len(false_positives) > 1:
            num_images = len(false_positives)
            _, axarr = plt.subplots(num_images, 1, figsize=(num_images, num_images))

            for i, fname in enumerate(false_positives):
                imname = fname.split('/')[-1]
                impath = os.path.join(self.no_obstacle_dataset, imname)
                im = cv2.imread(impath)
                im1 = im[:, :, 0]
                im2 = im[:, :, 1]
                im12 = cv2.hconcat([im1, im2])
                print(f'FP prediction: {preds[i]}, imname: {imname}')
                axarr[i].imshow(im12, cmap='gray', vmin=0, vmax=255)
                axarr[i].set_title(str(preds[i]) + ' ' + imname + (' '*30))
            plt.show()
        elif false_positives:
            fname = false_positives[0]
            imname = fname.split('/')[-1]
            impath = os.path.join(self.no_obstacle_dataset, imname)
            im = cv2.imread(impath)
            im1 = im[:, :, 0]
            im2 = im[:, :, 1]
            im12 = cv2.hconcat([im1, im2])
            plt.imshow(im12, cmap='gray', vmin=0, vmax=255)
            plt.title(str(preds[0]) + ' ' + imname)
            plt.show()

    def display_not_decided(self, predcitions, filenames, lower_threshold, upper_threshold):
        obstacle_image_names = [s.split('/')[-1] for s in self.obstacle_image_names]
        no_obstacle_image_names = [s.split('/')[-1] for s in self.no_obstacle_image_names]
        obs_title = ''
        
        not_decided = [fname for i, fname in enumerate(filenames)\
                       if ((lower_threshold < predictions[i] <= 0.5)\
                           and (fname.split('/')[-1] in no_obstacle_image_names))\
                           or ((0.5 <= predictions[i] < upper_threshold)\
                           and (fname.split('/')[-1] in obstacle_image_names))]
        
        preds = [p for i, p in enumerate(predictions)\
                       if ((lower_threshold < p <= 0.5)\
                           and (filenames[i].split('/')[-1] in no_obstacle_image_names))\
                           or ((0.5 <= p < upper_threshold)\
                           and (filenames[i].split('/')[-1] in obstacle_image_names))]

        if 500 > len(not_decided) > 1:
            num_images = len(not_decided)
            _, axarr = plt.subplots(num_images, 1, figsize=(1.5*num_images, 1.5*num_images))

            for i, fname in enumerate(not_decided):
                imname = fname.split('/')[-1]
                if imname in obstacle_image_names:
                    impath = os.path.join(self.obstacle_dataset, imname)
                    obs_title = 'obs'
                else:
                    impath = os.path.join(self.no_obstacle_dataset, imname)
                    obs_title = 'no_obs'
                im = cv2.imread(impath)
                im1 = im[:, :, 0]
                im2 = im[:, :, 1]
                im12 = cv2.hconcat([im1, im2])
                axarr[i].imshow(im12, cmap='gray', vmin=0, vmax=255)
                axarr[i].set_title(' '.join([imname, obs_title, str(preds[i]), (' '*30)]))
            plt.show()
        elif not_decided:
            imname = not_decided[0].split('/')[-1]
            if imname in obstacle_image_names:
                impath = os.path.join(self.obstacle_dataset, imname)
                obs_title = 'obs'
            else:
                impath = os.path.join(self.no_obstacle_dataset, imname)
                obs_title = 'no_obs'
            im = cv2.imread(impath)
            im1 = im[:, :, 0]
            im2 = im[:, :, 1]
            im12 = cv2.hconcat([im1, im2])
            print(f'ND prediction: {preds[0]}')
            plt.imshow(im12, cmap='gray', vmin=0, vmax=255)
            plt.title(' '.join([imname, obs_title, str(preds[0])]))
            plt.show()

    # An algorithm for optimal threshold taken from 
    # https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/
    # left here on case we consider re-using it.
    # Currently we are using the 2nd algorithm below
    def optimal_threshold_by_gmeans(self, num_no_obstacles, num_obstacles, predictions):
        true_values = [0]*num_no_obstacles + [1]*num_obstacles 
        fpr, tpr, thresholds = roc_curve(true_values, predictions)
        gmeans = sqrt(tpr * (1-fpr))
        ix = argmax(gmeans)

        return thresholds[ix]    
 
    # 2nd algorithm for finding optimal threshold, own developed
    # Currently used
    # Params: 
    #    -- predictions - a list of predicted values for no_obstacle / obstacle
    #    -- max_lost_ratio - a float [0.0-1.0], indicating the target maximum ratio (%/100)
    #    of ("not decided" / all) true values, for TP and TN - this is to avoid putting extreme 
    #    low/high thresholds, which will lower FN/FP, but will leave too few True values returned.
    #    This ratio ("not_decided" / all) is called here "lost ratio"
    # Steps:
    # 1. Set as (min_thresold / max_thresold) ==> (min(FN) / max(FP))
    #       Using this will perform with 0 FN/FP, 
    #       as there are neither FN below lower_thresold, not FP above upper_threshold,
    #       but --potentially-- with too many "not decided"
    # 2. If the lost ratio is too high - start lowering it to the target, using binary search
    #       to find the point where the ratio does not exceed the maximum lost-ration,
    #       while keeping the lower/upper thresholds as close as possible to their 
    #       original values. 
    def find_thresholds(self, predictions, max_lost_ratio=0.1):
        no_obs_preds = predictions[:handler.num_no_obstacles]
        obs_preds = predictions[handler.num_no_obstacles:]
        false_positives = np.array([p for p in no_obs_preds if p >= 0.5])
        false_negatives = np.array([p for p in obs_preds if p < 0.5])
        true_positives = np.array([p for p in obs_preds if p >= 0.5])
        true_negatives = np.array([p for p in no_obs_preds if p < 0.5])
        false_positives.sort(axis=0) 
        false_negatives.sort(axis=0)
        true_positives.sort(axis=0)
        true_negatives.sort(axis=0)

        true_positives_above = np.array([p for p in true_positives if p > max(false_positives)])
        true_negatives_below = np.array([p for p in true_negatives if p < min(false_negatives)])
        lost_true_positives = np.array([p for p in true_positives if p <= max(false_positives)])
        true_positives_lost_ratio = len(lost_true_positives) / len(true_positives)
        lost_true_negatives = np.array([p for p in true_negatives if p >= min(false_negatives)])
        true_negatives_lost_ratio = len(lost_true_negatives) / len(true_negatives)

        lower_threshold = min(false_negatives)
        lower_threshold_ind = np.where(true_negatives > lower_threshold)[0][0]
        num_true_negatives = len(true_negatives)
        gap = num_true_negatives - lower_threshold_ind
        
        # Binary search to find the point where no. of "not decided" satisfies the target

        if true_negatives_lost_ratio > max_lost_ratio:
            while gap:
                lost_ratio_diff = true_negatives_lost_ratio - max_lost_ratio
                if abs(lost_ratio_diff) < 0.01:
                    break
                gap //= 2
                if lost_ratio_diff < 0.0:
                    lower_threshold_ind -= gap
                else:
                    lower_threshold_ind += gap
                lost_true_negatives = true_negatives[lower_threshold_ind:]
                true_negatives_lost_ratio = len(lost_true_negatives) / num_true_negatives

            lower_threshold = true_negatives[lower_threshold_ind] 
            
        upper_threshold = max(false_positives)
        upper_threshold_ind = np.where(true_positives > upper_threshold)[0][0]
        num_true_positives = len(true_positives)
        gap = upper_threshold_ind

        # Binary search to find the point where no. of "not decided" satisfies the target
        
        if true_positives_lost_ratio > max_lost_ratio:
            while gap:
                lost_ratio_diff = true_positives_lost_ratio - max_lost_ratio
                if abs(lost_ratio_diff) < 0.01:
                    break
                gap //= 2
                if lost_ratio_diff < 0.0:
                    upper_threshold_ind += gap
                else:
                    upper_threshold_ind -= gap
                lost_true_positives = true_positives[:upper_threshold_ind]
                true_positives_lost_ratio = len(lost_true_positives) / num_true_positives

            upper_threshold = true_positives[upper_threshold_ind]
            
        return lower_threshold[0], upper_threshold[0]
    
    def write_metrics_to_csv(self, trial_component_display_name, print_params=True):
        client = boto3.client('sagemaker', region_name='eu-west-1')
        trials = client.list_trial_components()['TrialComponentSummaries']
        trial = [t for t in trials if t['DisplayName'] == trial_component_display_name]
        trial_description = None
        
        if trial:
            trial_component_name = trial[0]['TrialComponentName']
            trial_description = client.describe_trial_component(TrialComponentName=trial_component_name)

        metrics_values = [trial_component_display_name]        

        best_option = self.model_score.scored_models[self.model_score.best_option]
        metrics_values.append(best_option['score'])
        metrics_values.append(best_option['lost_true_values_percentage'])
        metrics_values.append(best_option['lower_threshold'])
        metrics_values.append(best_option['upper_threshold'])
        metrics_values.append(best_option['false_positives_percentage'])
        metrics_values.append(best_option['false_negatives_percentage'])
        
        if print_params:
            print(f'best option: {best_option["lost_true_values_percentage"]:.2f}')
            print(f'score: {best_option["score"]}')
            print(f'lower_threshold: {best_option["lower_threshold"]:.2f}')
            print(f'upper_threshold: {best_option["upper_threshold"]:.2f}')
            print(f'% FP: {best_option["false_positives_percentage"]:.2f}')
            print(f'%FN: {best_option["false_negatives_percentage"]:.2f}')

        if trial_description:
            metrics = trial_description['Metrics']

            metrics_prefixes = ['test', 'validation', '']
            metrics_separators = [' ', ' ', '']
            metrics_functions = ['auc', 'recall', 'specifity']        

            for i, metrics_prefix in enumerate(metrics_prefixes):
                for metrics_function in metrics_functions:
                    metric_name = metrics_separators[i].join([metrics_prefix, metrics_function])
                    metric_value = [m for m in metrics if m['MetricName'] == metric_name]
                    metrics_values.append(metric_value[0]['Max'])
                                
        self.metrics_csv_writer.writerow(metrics_values) 
        
    # Method to run when class is deleted
    def __del__(self):
        self.f_thresh_csv.close()
        self.f_metrics_csv.close()

In [3]:
class CloudDatasetHandler(DatasetHandler):
    
    def _get_random_dataset_image_names(self, dataset):
        client = boto3.client('s3')
        bucket = 'obstacles-classification'
        image_names = []

        paginator = client.get_paginator('list_objects')
        page_iterator = paginator.paginate(Bucket=bucket, Prefix=dataset)

        for page in page_iterator:
            for image_name in page['Contents']:
                if image_name['Key'].split('.')[-1] == 'jpg':
                    image_names.append(image_name['Key'])

        image_names = np.array(image_names)
        np.random.shuffle(image_names)
        image_names = image_names[:self.num_rand_images]

        return list(image_names)

    def _get_all_dataset_image_names(self, dataset):
        client = boto3.client('s3')
        bucket = 'obstacles-classification'
        image_names = []
        num_images = 0

        paginator = client.get_paginator('list_objects')
        page_iterator = paginator.paginate(Bucket=bucket, Prefix=dataset)

        for page in page_iterator:
            for image_name in page['Contents']:
                if image_name['Key'].split('.')[-1] == 'jpg':
                    image_names.append(image_name['Key'])
                    num_images += 1

        return image_names, num_images
            
    def _get_image(self, imname):
        client = boto3.client('s3')
        bucket = 'obstacles-classification'
        key = imname
        outfile = io.BytesIO()
        client.download_fileobj(bucket, key, outfile)
        outfile.seek(0)
        im = plt.imread(outfile, format='jpg')
        
        return im
    
    def get_metrics(self, model):
        return model.evaluate_generator(self.image_generator(), verbose=1)
    
    def get_predictions(self,model, color_mode='rgb'):
        return model.predict_generator(self.image_generator(), verbose=1)

In [4]:
class LocalDatasetHandler(DatasetHandler):
        
    def _get_random_dataset_image_names(self, dataset):
        image_names = os.listdir(dataset)
        image_paths = [os.path.join(dataset, image_name) for image_name in image_names]
        image_paths = np.array(image_paths)
        np.random.shuffle(image_paths)
        image_paths = image_paths[:self.num_rand_images]

        return list(image_paths)

    def _get_all_dataset_image_names(self, dataset):
        image_names = os.listdir(dataset)
        image_paths = [os.path.join(dataset, image_name) for image_name in image_names]
        
        return image_paths, len(image_paths)
            
    def _get_image(self, imname):
        imrgb = cv2.imread(imname)
        im = cv2.cvtColor(imrgb, cv2.COLOR_BGR2GRAY) 
        
        return im
    
    def get_metrics(self, model, color_mode='rgb'):
        if self.rand_images:
            return model.evaluate_generator(self.image_generator(), verbose=1)
        else:
            datagen = ImageDataGenerator(rescale=1./255)

            val_generator = datagen.flow_from_directory(
                self.dataset,
                target_size=(self.img_width, self.img_height),
                color_mode=color_mode,
                batch_size=self.batch_size,
                shuffle=False,
                class_mode='binary'
            )
            
            return model.evaluate_generator(val_generator, verbose=1)
        
    def get_predictions(self, model, color_mode='rgb'):
        if self.rand_images:
            return model.predict_generator(self.image_generator(), verbose=1)
        else:
            datagen = ImageDataGenerator(rescale=1./255)

            val_generator = datagen.flow_from_directory(
                self.dataset,
                target_size=(self.img_width, self.img_height),
                color_mode=color_mode,
                batch_size=self.batch_size,
                shuffle=False,
                class_mode='binary'
            )

            ret = model.predict_generator(val_generator, verbose=1)
            filenames = val_generator.filenames
            
            return ret, filenames

## Class to calculate model's score

In [5]:
class ModelScore():
    def __init__(self, alpha=0.7, beta=1.2, gamma=30):
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.max_score = 0.0
        self.scored_model_ind = 0
        self.scored_models = []
        
    '''
    Cliff shape function
    
    Based on the Sigmoid function:
        1/(1 + np.exp(-x))

    with:
    1. Changing (x) to (1-x), so we flip the curve to be high near 0 and decline sharply at some point, till reaching zero
    2. Adding alpha, beta and gamma modifiers to enable controlling the curve's attributes:
        * alpha controls the point where the graph starts to decline sharply (= the % of lost true values which is berable and should get relatively high score for this aspect)
        * beta controls the width of the sharply declining portion of the curve
        * gamma controls the smoothness of the cliff-shape part of the graph. A high gamma will make the curvie less somooth, i.e. more 'cliffy'. 

    So our Sigmoid-modofied function is:

        1/(1 + np.exp(-(1-(x+alpha)*beta)*gamma)) 
    '''
    def cliff(self, x):
        return 1/(1 + np.exp(-(1-(x+self.alpha)*self.beta)*self.gamma)) 
    
    # A method to insert into overall model's scores an option, regarding 
    # the lost_true_value_percentage. 
    # The goal is that the class remembers the score attached to this option,
    # then can compare all scores and find the best one
    def add_model_option(self,
                         lost_true_values,
                         lost_true_values_percentage,
                         false_positives,
                         false_positives_percentage,
                         false_negatives,
                         false_negatives_percentage,
                         lower_threshold,
                         upper_threshold,
                         predictions):
        
        s1 = self.cliff(lost_true_values_percentage)
        s2 = 1.0 - (false_positives_percentage + false_negatives_percentage)
        score = (s1 + s2) / 2
        if score > self.max_score:
            self.max_score = score
            self.best_option = self.scored_model_ind
        scored_model = {}
        scored_model['score'] = score
        scored_model['lost_true_values'] = lost_true_values
        scored_model['lost_true_values_percentage'] = lost_true_values_percentage
        scored_model['false_positives'] = false_positives
        scored_model['false_positives_percentage'] = false_positives_percentage
        scored_model['false_negatives'] = false_negatives
        scored_model['false_negatives_percentage'] = false_negatives_percentage
        scored_model['lower_threshold'] = lower_threshold
        scored_model['upper_threshold'] = upper_threshold
        scored_model['predictions'] = predictions
        self.scored_models.append(scored_model)
        self.scored_model_ind += 1

## Main

(Need special handling of the case number of FP == 0. Currently fails in this scenario.)

In [6]:
# -- Variables --
trial_component_display_name = 'gg-2-const-2021-08-25-14-17-49-training-trial'
model_path = '/home/drevital/cs_video_processor/models/new_factory_gg_2_const'
cloud_dataset = False
obstacle_dataset = '/home/drevital/obstacles_classification_datasets/new_factory_gg_2_const/eval/obstacle'
no_obstacle_dataset = '/home/drevital/obstacles_classification_datasets/new_factory_gg_2_const/eval/no_obstacle'
model_name = model_path.split('/')[-1]
color_mode = 'rgb'
#thresh_csv_fname = f'model_thresholds.{model_name}.csv'
#metrics_csv_fname = f'model_metrics.{model_name}.csv'
thresh_csv_fname = None
metrics_csv_fname = None
rand_images = False
num_rand_images = 10
print_cm_params=True
display_cm_figure = False
display_false_negatives = False
display_false_positives = False
display_not_decided = False
print_best_option_params=False

# -- Run the Evaluation --
model = tf.keras.models.load_model(model_path)
dataset_handlers = [LocalDatasetHandler, CloudDatasetHandler]
handler = dataset_handlers[cloud_dataset](model_path,
                                          obstacle_dataset,
                                          no_obstacle_dataset,
                                          rand_images=rand_images,
                                          num_rand_images=num_rand_images,
                                          thresh_csv_fname=thresh_csv_fname,
                                          metrics_csv_fname=metrics_csv_fname)

# -- Print metrics
metrics = handler.print_model_metrics(model, color_mode)
labels = np.array([0]*(handler.num_no_obstacles) + [1]*(handler.num_obstacles))

# -- Predict with the model
predictions, filenames = handler.get_predictions(model, color_mode=color_mode)

# -- Print confusion-matrix considering the older "no decision" thresholds
threshold = 0.5
decision_margin = 0.25
lower_threshold = threshold * (1.00 - decision_margin)
upper_threshold = threshold + decision_margin * (1.00 - threshold)
handler.calc_cm_considering_thresholds(model_path,
                                       labels,
                                       predictions,
                                       lower_threshold,
                                       upper_threshold,
                                       print_params=print_cm_params,
                                       plot=display_cm_figure,
                                       threshold=threshold)

# Find optimal lower, upper thresholds per given max limit of "not decided" ("lost") predictions
max_lost_ratios = np.arange(start=0.005, stop=0.205, step=0.005)

print('Experimenting best max_ratio ...')
for max_lost_ratio in max_lost_ratios:
    lower_threshold, upper_threshold = handler.find_thresholds(predictions,
                                                               max_lost_ratio=max_lost_ratio)
    print(f'{max_lost_ratio:.4f}', end='...')
    # Print confusion-matrix after using recommended thresholds
    handler.calc_cm_considering_thresholds(model_path,
                                           labels,
                                           predictions,
                                           lower_threshold,
                                           upper_threshold,
                                           max_lost_ratio=max_lost_ratio,
                                           threshold=0.5,
                                           print_params=print_cm_params,
                                           plot=display_cm_figure,
                                           print_to_csv=False)
    
best_option = handler.model_score.scored_models[handler.model_score.best_option]

print('\n\n')
print(f'best option score: {best_option["score"]}')
print(f'best option lower threshold: {best_option["lower_threshold"]}')
print(f'best option upper threshold: {best_option["upper_threshold"]}')
print(f'best option lost true values: {best_option["lost_true_values"]}', end = ' ')
print(f'({best_option["lost_true_values_percentage"]*100:.2f}%)')
print(f'best option false negatives: {best_option["false_negatives"]}', end=' ')
print(f'({best_option["false_negatives_percentage"]*100:.2f}%)')
print(f'best option false positives: {best_option["false_positives"]}', end=' ')
print(f'({best_option["false_positives_percentage"]*100:.2f}%)')

lower_threshold = best_option['lower_threshold']
upper_threshold = best_option['upper_threshold']
    
if display_false_negatives:
    print('\nFALSE NEGATIVES\n')
    handler.display_false_negatives(predictions, filenames, lower_threshold)

if display_false_positives:
    print('\nFALSE POSITIVES\n')
    handler.display_false_positives(predictions, filenames, upper_threshold)

if display_not_decided:
    print('\nNOT DECIDED\n')
    handler.display_not_decided(predictions, filenames, lower_threshold, upper_threshold)

print()
    
# Retrieve model's metrics from SageMaker and write them to the handler's .csv file
handler.write_metrics_to_csv(trial_component_display_name,
                             print_params=print_best_option_params)
del handler

Found 2400 images belonging to 2 classes.


/home/drevital/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


75/75 [==============================] - 14s 174ms/step - loss: 0.3876 - auc: 0.9982 - recall: 0.9975 - specifity: 0.9992
loss :  0.3875793516635895
auc :  0.9982079267501831
recall :  0.9975000023841858
specifity :  0.9991666674613953
Found 2400 images belonging to 2 classes.


/home/drevital/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


75/75 [==============================] - 13s 172ms/step

No Obstacles Detected (True Negatives): 1161 (48.38%)
No Obstacles Incorrectly Detected (False Positives): 18 (0.75%)
Obstacles Missed (False Negatives): 2 (0.08%)
Obstacles Detected (True Positives): 1191 (49.62%)
Not Decided: 28 (1.17%)
True Not Decided: 21 (0.88%)
Total Obstacles: 2400
Experimenting best max_ratio ...
0.0050...
Max Lost Ratio: 0.005
No Obstacles Detected (True Negatives): 1162 (48.42%)
No Obstacles Incorrectly Detected (False Positives): 9 (0.38%)
Obstacles Missed (False Negatives): 2 (0.08%)
Obstacles Detected (True Positives): 1180 (49.17%)
Not Decided: 47 (1.96%)
True Not Decided: 29 (1.22%)
Total Obstacles: 2400
0.0100...
Max Lost Ratio: 0.01
No Obstacles Detected (True Negatives): 1162 (48.42%)
No Obstacles Incorrectly Detected (False Positives): 9 (0.38%)
Obstacles Missed (False Negatives): 2 (0.08%)
Obstacles Detected (True Positives): 1180 (49.17%)
Not Decided: 47 (1.96%)
True Not Decided: 29 (1.22%)
T

0.1300...
Max Lost Ratio: 0.13
No Obstacles Detected (True Negatives): 1017 (42.38%)
No Obstacles Incorrectly Detected (False Positives): 1 (0.04%)
Obstacles Missed (False Negatives): 2 (0.08%)
Obstacles Detected (True Positives): 1037 (43.21%)
Not Decided: 343 (14.29%)
True Not Decided: 317 (13.36%)
Total Obstacles: 2400
0.1350...
Max Lost Ratio: 0.135
No Obstacles Detected (True Negatives): 1017 (42.38%)
No Obstacles Incorrectly Detected (False Positives): 1 (0.04%)
Obstacles Missed (False Negatives): 2 (0.08%)
Obstacles Detected (True Positives): 1037 (43.21%)
Not Decided: 343 (14.29%)
True Not Decided: 317 (13.36%)
Total Obstacles: 2400
0.1400...
Max Lost Ratio: 0.14
No Obstacles Detected (True Negatives): 1017 (42.38%)
No Obstacles Incorrectly Detected (False Positives): 1 (0.04%)
Obstacles Missed (False Negatives): 2 (0.08%)
Obstacles Detected (True Positives): 1037 (43.21%)
Not Decided: 343 (14.29%)
True Not Decided: 317 (13.36%)
Total Obstacles: 2400
0.1450...
Max Lost Ratio: 0

AttributeError: 'LocalDatasetHandler' object has no attribute 'metrics_csv_writer'